# First MSSP example

In [1]:
import gurobipy as gb
from gurobipy import GRB
import pandas as pd
import numpy as np
import math
from collections import namedtuple

## Manage datas

Read the synthetic instance of Section 3.2

In [2]:
nodes_number = 25
agents_number = 4
synthetic_5x5_df = pd.read_csv("data/d_it_ij_5x5_1it.csv",
                               header=0,
                               names=["i", "j", "d_ij"])

# convert d_ij column into float
# print(synthetic_5x5_df.dtypes)
synthetic_5x5_df["d_ij"] = synthetic_5x5_df["d_ij"].str.replace(
    ",", ".").astype(np.float64)
# print(synthetic_5x5_df.dtypes)

# counting network nodes starting from 0
synthetic_5x5_df[["i", "j"]] = synthetic_5x5_df[["i", "j"]] - 1


Set up datas that will be used to solve the problem

In [3]:
Arc = namedtuple("Arc", synthetic_5x5_df.columns)
arcs = [Arc(*row) for row in synthetic_5x5_df.itertuples(index=False)]  # as List to avoid nusty bugs

nodes = np.unique(np.concatenate((synthetic_5x5_df["i"].to_numpy(),
                              synthetic_5x5_df["j"].to_numpy())))
agent_sources = np.array([0, 2, 3, 4])
agent_terminus = np.array([20, 22, 23, 24])

agents = range(agents_number)
# D = synthetic_5x5_df["d_ij"].to_numpy()

## Manage the problem

Create the problem

In [4]:
MSPP_pb = gb.Model("First MSPP")
MSPP_pb.setParam("OutputFlag", 0)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-10


Define decision variables

In [5]:
var_shape = len(nodes), len(nodes), agents_number

X = MSPP_pb.addMVar(var_shape,
                    vtype=GRB.BINARY,  # 5) Binary constraints
                    name="k-th agent traverse arc (i,j)")


Define the objective function

In [6]:
# 1-3) Objective function
MSPP_pb.setObjective(
    gb.quicksum(
        arc.d_ij*X[arc.i, arc.j, k]
        for arc in arcs for k in agents
    ),
    GRB.MINIMIZE
)

Add constraints

In [7]:
# 4) Flow constraints

def compute_flow(X, node, arcs, agent):
    flow_out = gb.quicksum(X[node, arc.j, agent] for arc in arcs if arc.i==node)
    flow_in = gb.quicksum(X[arc.i, node, agent] for arc in arcs if arc.j==node)
    return flow_out - flow_in


for k in agents:
    for i in nodes:
        if i == agent_sources[k]:
            MSPP_pb.addConstr(compute_flow(X, i, arcs, k) == 1,
            name=f"Flow constr related for agent {k} in node {i}")
        elif i == agent_terminus[k]:
            MSPP_pb.addConstr(compute_flow(X, i, arcs, k) == -1,
            name=f"Flow constr related for agent {k} in node {i}")
        else:
            MSPP_pb.addConstr( compute_flow(X, i, arcs, k) == 0,
            name=f"Flow constr related for agent {k} in node {i}")
            

Solve the problem

In [8]:
MSPP_pb.optimize()

Report results

In [14]:
print("Result of the optimization is:")
if MSPP_pb.Status == 2:
    print("optimal")
elif MSPP_pb.Status == 3:
    print("infeasible")
elif MSPP_pb.Status == 5:
    print("unbounded")
else:
    print("Some other return status")

Result of the optimization is:
optimal
Agent 0 will follow the path:
Arc(i=0, j=6, d_ij=1.0) -> 
Arc(i=6, j=12, d_ij=1.0) -> 
Arc(i=12, j=16, d_ij=1.0) -> 
Arc(i=16, j=20, d_ij=1.0) -> 
Agent 1 will follow the path:
Arc(i=2, j=7, d_ij=1.0) -> 
Arc(i=7, j=12, d_ij=1.0) -> 
Arc(i=12, j=17, d_ij=1.0) -> 
Arc(i=17, j=22, d_ij=1.0) -> 
Agent 2 will follow the path:
Arc(i=3, j=7, d_ij=1.0) -> 
Arc(i=7, j=12, d_ij=1.0) -> 
Arc(i=12, j=17, d_ij=1.0) -> 
Arc(i=17, j=23, d_ij=1.0) -> 
Agent 3 will follow the path:
Arc(i=4, j=8, d_ij=1.0) -> 
Arc(i=8, j=14, d_ij=1.0) -> 
Arc(i=14, j=18, d_ij=1.0) -> 
Arc(i=18, j=24, d_ij=1.0) -> 
Minimum distance covered is 16.0


In [ ]:
print(f"Minimum distance covered is {MSPP_pb.ObjVal}")

In [22]:
for k in agents:
    print(f"Agent {k} will follow the path:")
    for arc in arcs:
        if math.isclose(X.X[arc.i, arc.j, k], 1):
            print(f"{arc.i}->{arc.j}", end="\t")
    print()


Agent 0 will follow the path:
0->6	6->12	12->16	16->20	
Agent 1 will follow the path:
2->7	7->12	12->17	17->22	
Agent 2 will follow the path:
3->7	7->12	12->17	17->23	
Agent 3 will follow the path:
4->8	8->14	14->18	18->24	
